In [ ]:
import shapefile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import gdal
import cv2
import copy

# Helper Functions

In [ ]:
def draw_circles(img, circle_centers, radius = 20, line_thickness = 3):
    img_with_circles = copy.deepcopy(img)
    
    for circle_center in circle_centers:
        cv2.circle(img_with_circles, 
                   circle_center, 
                   radius=radius,
                   color=(255,99,71), 
                   thickness=line_thickness)
    return img_with_circles

# Read Shapefile

In [ ]:
shape = shapefile.Reader("data/manually_picked_geese.shp")
features = shape.shapeRecords()
geeze_coordinates = [feature.shape.__geo_interface__['coordinates'] for feature in features]
geeze_coordinates

# Read Image

In [ ]:
img = mpimg.imread('data/Nunavut_17Juin_2020_30cm_RGBN_W84U17_1_3_clipped.tif')
img.shape

# Read Geo Transformation Matrix from GeoTif

In [ ]:
ds = gdal.Open('data/Nunavut_17Juin_2020_30cm_RGBN_W84U17_1_3_clipped.tif') 
xoffset, px_w, rot1, yoffset, px_h, rot2 = ds.GetGeoTransform()
gt = ds.GetGeoTransform()

In [ ]:
def convert_to_image_coordinates(goose_coordinates):
    x, y = goose_coordinates
    return (int((x - gt[0]) / gt[1]), int((y - gt[3]) / gt[5]))

# Plotting region with geeze marked

In [ ]:
img = mpimg.imread('data/Nunavut_17Juin_2020_30cm_RGBN_W84U17_1_3_clipped.tif')

# convert coordinates into pixel-coordinates of image
image_geeze_coords = [convert_to_image_coordinates(geeze_coordinate) for geeze_coordinate in geeze_coordinates]
# using draw_circles function defined on top
img_with_geeze_marks = draw_circles(img, image_geeze_coords)

plt.figure(figsize=(20,10))
plt.imshow(img_with_geeze_marks[2600:5500, 5000:9000])

# Searching for geeze using openCV

comments in code.

In [ ]:
# 1. Read Image again
img = mpimg.imread('data/Nunavut_17Juin_2020_30cm_RGBN_W84U17_1_3_clipped.tif')

# 2. Convert to Greyscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 3. Binarize Image
# ToDo: Optimize binarization
ret,thresh = cv2.threshold(gray,120,255,cv2.THRESH_BINARY)

In [ ]:
# Getting coordinates of a goose to crop sample image region
sample_goose_id = 2
x_coord, y_coord = convert_to_image_coordinates(geeze_coordinates[sample_goose_id])

# Cropping image around this sample goose
preprocessed_roi = thresh[y_coord-100:y_coord+100, x_coord-100:x_coord+100]
original_roi = img[y_coord-100:y_coord+100, x_coord-100:x_coord+100]

# plotting :-)
f, axarr = plt.subplots(1,2,figsize=(20,20))
axarr[0].imshow(original_roi)
axarr[1].imshow(preprocessed_roi)

In [ ]:
def find_geeze(roi_image, area_threshold=2):
    """finds geeze in image area

    Keyword arguments:
    roi_image -- image to search for geeze.
    area_threshold -- threshold of which size a cluster of pixels is marked as goose
    """
    (contours, _) = cv2.findContours(roi_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detected_geeze = []
    for contour in contours: 
        area = cv2.contourArea(contour)
        if area <= area_threshold:
            detected_geeze.append({
                'upper_left_coord': (contour[0][0][0], contour[0][0][1]),
                'area': area                      
            })
    return detected_geeze

In [ ]:
# testing the find_geeze function
geeze = find_geeze(preprocessed_roi)

In [ ]:
geeze

In [ ]:
result = draw_circles(preprocessed_roi, 
                      [goose['upper_left_coord'] for goose in geeze],
                      radius = 5,
                      line_thickness = 1
                     )
    
# plotting :-)
f, axarr = plt.subplots(1,2,figsize=(20,20))
axarr[0].imshow(draw_circles(img, image_geeze_coords, radius = 5, line_thickness = 1)[y_coord-100:y_coord+100, x_coord-100:x_coord+100])
axarr[1].imshow(result)

In [ ]:
original_roi_with_circles = draw_circles(original_roi, 
                      [goose['upper_left_coord'] for goose in geeze],
                      radius = 5,
                      line_thickness = 1
                     )

plt.figure(figsize=(30,30))
plt.imshow(original_roi_with_circles)

# Playground

In [ ]:
img = cv2.imread('data/spectral_analysis_geese_clipped.tif',-1)
cv2.imshow('tiff',img)
cv2.waitKey(0)

In [ ]:
img = mpimg.imread('data/spectral_analysis_geese_clipped.tif')
plt.imshow(img)

In [ ]:
def converte_x(pos_y):
    return (pos_y - yoffset)/rot2

def converte_y(pos_x, pos_y):
    return pos_x-px_w*converte_x(pos_y)-xoffset